# 📘 Word → CSV → YAML → 文档生成 Pipeline Demo
本 Notebook 演示如何将 **非结构化的 Word 文档（AT 命令集）** 转化为结构化数据，最终生成 Markdown 文档。

流程：Word → CSV → YAML → Markdown/PDF

In [1]:
# 安装依赖（首次运行时需要）
!pip install python-docx pandas pyyaml jinja2

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 5.6 MB/s  0:00:01 eta 0:00:01
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 7.6 MB/s  0:00:01 eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 8.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [jinja2] 9/10 [jinja2]docx]


## Step 1. 从 Word 提取表格 → CSV
假设 Word 文档里有一张表格：指令 | 描述 | 示例响应。
我们用 python-docx 提取并存成 CSV。

In [6]:
import pandas as pd
from docx import Document

# 示例：我们假设有个 word 文件 demo.docx
# 实际运行时换成你的 Word 文档路径
doc = Document("de.docx")
table = doc.add_table(rows=3, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text, hdr_cells[1].text, hdr_cells[2].text = '指令', '描述', '示例响应'
table.rows[1].cells[0].text, table.rows[1].cells[1].text, table.rows[1].cells[2].text = 'AT+CSQ', '查询信号强度', '+CSQ: 25,0'
table.rows[2].cells[0].text, table.rows[2].cells[1].text, table.rows[2].cells[2].text = 'AT+CGATT', '网络附着', '+CGATT: 1'

demo_path = "demo.docx"
doc.save(demo_path)

# 重新读取并提取表格
doc = Document(demo_path)
data = []
for table in doc.tables:
    keys = None
    for i, row in enumerate(table.rows):
        text = [cell.text.strip() for cell in row.cells]
        if i == 0:
            keys = text
        else:
            data.append(dict(zip(keys, text)))

df = pd.DataFrame(data)
csv_file = "commands.csv"
df.to_csv(csv_file, index=False, encoding="utf-8")
df

,类型,命令,响应格式,<manufacturer>,模组厂商信息、产品名称、版本号,命令。,<signal>,以下为signal(CSQ)与rssi对应关系：,<n>,0：禁止网络注册主动提供结果代码（默认设置）\n1：允许网络注册主动提供结果代码\n2：允许网络注册主动提供所在地讯息（CELL ID、LOCAL ID）。,...,设置短信中心号码，格式为国际号码类型,<fo>,<fo>：取决于该命令或结果码：GSM 03.40 SMS-DELIVER的前8位；\nSMS-SUBMIT(缺省值：17)；或采用整数型的SMS-COMMAND（缺省值：2）,"AT+CSMP=17,167,0,0\n\nOK","设置文本模式参数为17,167,0,0；\n表示没有状态报告回报，信息有效期时间为24小时，只能发送文本格式短信","AT+CSDH=0\n\nOK\nAT+CMGR=0\n\n+CMGR: ""REC READ"",""13510895077"",,""15/07/23,20: 58: 28\n+32""\nabc\nOK",设置不显示详细的头信息\n读取第0条短信,指令,描述,示例响应
0,执行,ATI<CR>,<CR><LF><manufacturer>\n<CR><LF><module_versio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,<module_version>,模组型号,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,<soft_version>,模组软件版本,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,执行,AT+GMR<CR>,<CR><LF>+GMR: <reversion>\n<CR><LF>OK<CR><LF>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,执行,NaN,"<CR><LF>+CSQ: <signal>,<ber>\n<CR><LF>OK<CR><LF>",NaN,NaN,AT+CSQ<CR>,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"AT+CSDH=1\n\nOK\nAT+CMGR=0\n\n+CMGR: ""REC READ...",设置显示详细的头信息\n读取第0条短信,NaN,NaN,NaN
269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,AT+CSDH?\n\n+CSDH: 0\nOK,查询指令当前设置参数,NaN,NaN,NaN
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"AT+CSDH=?\n\n+CSDH: (0,1)\nOK",查询当前指令取值范围,NaN,NaN,NaN
271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AT+CSQ,查询信号强度,"+CSQ: 25,0"


## Step 2. 从 CSV 转换为 YAML
用 Pandas 读取 CSV，再输出成结构化 YAML。

In [3]:
import yaml

df = pd.read_csv(csv_file)
records = df.to_dict(orient="records")

yaml_data = {"product": "N706B", "version": "V1.0", "commands": records}
yaml_file = "commands.yaml"
with open(yaml_file, "w", encoding="utf-8") as f:
    yaml.dump(yaml_data, f, allow_unicode=True)

print(open(yaml_file, encoding="utf-8").read())

commands:
- 指令: AT+CSQ
  描述: 查询信号强度
  示例响应: '+CSQ: 25,0'
- 指令: AT+CGATT
  描述: 网络附着
  示例响应: '+CGATT: 1'
product: N706B
version: V1.0



## Step 3. 用 Jinja2 模板生成 Markdown
我们利用结构化的 YAML 数据，统一渲染成 Markdown 文档。

In [ ]:
from jinja2 import Template

template_str = """
# {{ product }} 模块 AT 命令手册 (版本 {{ version }})

| 指令   | 描述   | 示例响应 |
|--------|--------|----------|
{% for c in commands %}| {{ c['指令'] }} | {{ c['描述'] }} | {{ c['示例响应'] }} |
{% endfor %}
"""
template = Template(template_str)

output_md = template.render(**yaml_data)
md_file = "AT_manual.md"
with open(md_file, "w", encoding="utf-8") as f:
    f.write(output_md)
print(output_md)

## Step 4. 下一步拓展
- 现在你已经能跑通 Word → CSV → YAML → Markdown。
- 可以接 Pandoc：
  ```bash
  pandoc AT_manual.md -o AT_manual.pdf
  ```
- 或者接 Sphinx / DITA-OT 做大规模文档管理。
- 更进一步：在 CI/CD 流水线里，把 Word 自动化转换为结构化数据 → 渲染 → 发布。